# Observations
1.	Charter schools on average had higher overall scores than that of district schools
2.	Smaller schools (less than 2,000 students) had higher average scores in reading and math.
3.	More spending per student did not necessarily result in better scores.  On average schools spending less 
than $625 per student had higher overall scores.


In [1]:
import pandas as pd
import numpy as np

In [2]:
#Create District Summary
file1 = '../10-16-2017-GW-Arlington-Class-Repository-DATA/Homework/04-Numpy-Pandas/Instructions/PyCitySchools/raw_data/schools_complete.csv'
file2 = '../10-16-2017-GW-Arlington-Class-Repository-DATA/Homework/04-Numpy-Pandas/Instructions/PyCitySchools/raw_data/students_complete.csv'
df_sch = pd.read_csv(file1,low_memory=False)
df_stu = pd.read_csv(file2, low_memory = False)
df_sch = df_sch.rename(columns={'name':'school'})
df_sch['budg_stu'] = df_sch['budget']/df_sch['size']
df_sch.head()
df_stu.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [3]:
#Key metrics
#School count
school = df_sch["school"].count()
#Total Students
students = df_sch["size"].sum()
#Total Budget
budget = df_sch["budget"].sum()
#Average Math Scoore
avg_math = df_stu["math_score"].mean()
#Average Reading Score
avg_read = df_stu['reading_score'].mean()
#% passing math
per_math = df_stu.loc[df_stu['math_score']>=60]
pass_math = per_math['math_score'].count()
pm = pass_math/students*100
#% passing reading
per_read = df_stu.loc[df_stu['reading_score']>=60]
pass_read = per_read['reading_score'].count()
pr = pass_read/students*100
#Overall Passing Rate
poverall = (pm+pr)/2

In [4]:
#District Summary
df_summary = pd.DataFrame({'school_count':[school],
                           'student_count':[students],
                           'total_budget':[budget],
                           'avg_math_score':[avg_math],
                           'avg_reading_score':[avg_read],
                           'percent_passing_math':[pm],
                           'percent_passing_reading':[pr],
                           'percent_passing_overall':[poverall]
                          })
df_summary.round(2)

,avg_math_score,avg_reading_score,percent_passing_math,percent_passing_overall,percent_passing_reading,school_count,student_count,total_budget
0,78.99,81.88,92.45,96.22,100.0,15,39170,24649428


In [5]:
#per school summary - narrow scope of dataframe
df_stu1 = df_stu[['school','reading_score','math_score']]

#Use counts to get number of passing
df_stu1['reading_pass_count'] = [1 if x >=60 else 0 for x in df_stu1['reading_score']]
df_stu1['math_pass_count'] = [1 if x >=60 else 0 for x in df_stu1['math_score']]

#Group by school to get totals and averages
df_stu1 = df_stu1.groupby('school')
stu_pass = df_stu1['reading_pass_count','math_pass_count'].sum()
stu_pass = stu_pass.reset_index()
stu_avg = df_stu1['reading_score','math_score'].mean()
stu_avg = stu_avg.reset_index()

#merge dataframes to get data into 1 table
df_merge = pd.merge(stu_pass, stu_avg, on='school', how='outer')

#rename columns
df_merge = df_merge.rename(columns={'reading_score':'avg_reading_score','math_score':'avg_math_score'})
df_merge

C:\Users\menta\Anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\menta\Anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,school,reading_pass_count,math_pass_count,avg_reading_score,avg_math_score
0,Bailey High School,4976,4455,81.033963,77.048432
1,Cabrera High School,1858,1858,83.975780,83.061895
2,Figueroa High School,2949,2608,81.158020,76.711767
3,Ford High School,2739,2446,80.746258,77.102592
4,Griffin High School,1468,1468,83.816757,83.351499
5,Hernandez High School,4635,4129,80.934412,77.289752
6,Holden High School,427,427,83.814988,83.803279
7,Huang High School,2917,2592,81.182722,76.629414
8,Johnson High School,4761,4246,80.966394,77.072464
9,Pena High School,962,962,84.044699,83.839917


In [6]:
df_merge1 = pd.merge(df_merge,df_sch, on='school', how='outer')

#Get percentages passing
df_merge1['reading_pass_per'] = (df_merge1['reading_pass_count']/df_merge1['size'])*100
df_merge1['math_pass_per'] = (df_merge1['math_pass_count']/df_merge1['size'])*100
df_merge1.head()

#Summary School Table
df_summary = df_merge1.iloc[:,[0,6,7,8,9,3,4,10,11]]
df_summary['overall_pass_per'] = (df_summary['reading_pass_per']+df_summary['math_pass_per'])/2
df_summary = df_summary.rename(columns={'school':'school_name','size':'total_students','budget':'total_budget',
                                       'budg_stu':'budget_per_student'
                                       }
                              )
df_summary

C:\Users\menta\Anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,school_name,type,total_students,total_budget,budget_per_student,avg_reading_score,avg_math_score,reading_pass_per,math_pass_per,overall_pass_per
0,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,100.0,89.529743,94.764871
1,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,100.0,100.000000,100.000000
2,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,100.0,88.436758,94.218379
3,Ford High School,District,2739,1763916,644.0,80.746258,77.102592,100.0,89.302665,94.651333
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,100.0,100.000000,100.000000
5,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,100.0,89.083064,94.541532
6,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,100.0,100.000000,100.000000
7,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,100.0,88.858416,94.429208
8,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,100.0,89.182945,94.591472
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,100.0,100.000000,100.000000


In [7]:
#Top performing schools by overall passing rate
df_summary.sort_values(by=['overall_pass_per'], ascending = False, inplace = True)
df_summary.head(5)


,school_name,type,total_students,total_budget,budget_per_student,avg_reading_score,avg_math_score,reading_pass_per,math_pass_per,overall_pass_per
1,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,100.0,100.0,100.0
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,100.0,100.0,100.0
6,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,100.0,100.0,100.0
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,100.0,100.0,100.0
11,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,100.0,100.0,100.0


In [8]:
#Bottom performing schools by overall passing rate
df_summary.sort_values(by=['overall_pass_per'], ascending = True, inplace = True)
df_summary.head(5)

,school_name,type,total_students,total_budget,budget_per_student,avg_reading_score,avg_math_score,reading_pass_per,math_pass_per,overall_pass_per
2,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,100.0,88.436758,94.218379
10,Rodriguez High School,District,3999,2547363,637.0,80.744686,76.842711,100.0,88.547137,94.273568
7,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,100.0,88.858416,94.429208
5,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,100.0,89.083064,94.541532
8,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,100.0,89.182945,94.591472


In [9]:
#Reading scores by grade by school

df_stugrp = df_stu[['school','grade','reading_score','math_score']]

#group by school, grade
df_stugrp = df_stugrp.groupby(['school','grade'])

#Average score
df_stugrp1 = pd.DataFrame(df_stugrp['reading_score'].mean())
df_stugrp1 = df_stugrp1.reset_index()

#Rename column
df_stugrp1 = df_stugrp1.rename(columns = {'reading_score':'avg_reading_score'})

#Sort data
df_stugrp1.sort_values(by=['school','grade'],ascending=['school','grade'])
df_stugrp1

,school,grade,avg_reading_score
0,Bailey High School,10th,80.907183
1,Bailey High School,11th,80.945643
2,Bailey High School,12th,80.912451
3,Bailey High School,9th,81.303155
4,Cabrera High School,10th,84.253219
5,Cabrera High School,11th,83.788382
6,Cabrera High School,12th,84.287958
7,Cabrera High School,9th,83.676136
8,Figueroa High School,10th,81.408912
9,Figueroa High School,11th,80.640339


In [10]:
#Math scores by grade by school

#Average score
df_stugrp2 = pd.DataFrame(df_stugrp['math_score'].mean())
df_stugrp2 = df_stugrp2.reset_index()

#Rename column
df_stugrp2 = df_stugrp2.rename(columns = {'math_score':'avg_math_score'})

#Sort data
df_stugrp2.sort_values(by=['school','grade'],ascending=['school','grade'])
df_stugrp2

,school,grade,avg_math_score
0,Bailey High School,10th,76.996772
1,Bailey High School,11th,77.515588
2,Bailey High School,12th,76.492218
3,Bailey High School,9th,77.083676
4,Cabrera High School,10th,83.154506
5,Cabrera High School,11th,82.765560
6,Cabrera High School,12th,83.277487
7,Cabrera High School,9th,83.094697
8,Figueroa High School,10th,76.539974
9,Figueroa High School,11th,76.884344


In [11]:
#breakdown School performance by Spending Ranges per Student
df_summary
bins = [575,600,625,650,675]
groupnames = ['>575_<600','>600_<625','>625_<650','>650_<675']

#binning the summary table
df_summarybin = df_summary[['budget_per_student','avg_reading_score','avg_math_score',
                           'reading_pass_per','math_pass_per','overall_pass_per']]
df_summarybin['spending_range'] = pd.cut(df_summary['budget_per_student'],bins,labels = groupnames)

#Group by Spending Range
spending_range = df_summarybin.groupby(['spending_range'])
spending_range.mean()

C:\Users\menta\Anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,budget_per_student,avg_reading_score,avg_math_score,reading_pass_per,math_pass_per,overall_pass_per
spending_range,,,,,,
>575_<600,584.800000,83.892196,83.436210,100.0,100.000000,100.000000
>600_<625,617.000000,83.930728,83.595708,100.0,100.000000,100.000000
>625_<650,639.333333,81.416375,78.032719,100.0,90.833208,95.416604
>650_<675,653.500000,81.058567,76.959583,100.0,88.970740,94.485370


In [12]:
#Summary table for School size
#breakdown School performance by School Size
bins = [0,2000,4000,6000]
groupnames = ['small','medium','large']

#binning the summary table
df_summarybin2 = df_summary[['total_students','avg_reading_score','avg_math_score',
                           'reading_pass_per','math_pass_per','overall_pass_per']]
df_summarybin2['school_size'] = pd.cut(df_summary['total_students'],bins,labels = groupnames)

#Group by School Size
school_size = df_summarybin2.groupby(['school_size'])
school_size.mean()

C:\Users\menta\Anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,total_students,avg_reading_score,avg_math_score,reading_pass_per,math_pass_per,overall_pass_per
school_size,,,,,,
large,4790.666667,80.978256,77.136883,100.0,89.265250,94.632625
medium,2977.400000,81.564235,78.112137,100.0,91.028995,95.514498
small,1415.857143,83.883125,83.502373,100.0,100.000000,100.000000


In [13]:
#Summary table by Type
df_summarybin3 = df_summary[['type','avg_reading_score','avg_math_score',
                           'reading_pass_per','math_pass_per','overall_pass_per']]

#Group by School Type
school_type = df_summarybin3.groupby(['type'])
school_type.mean()

,avg_reading_score,avg_math_score,reading_pass_per,math_pass_per,overall_pass_per
type,,,,,
Charter,83.896421,83.473852,100.0,100.000000,100.000000
District,80.966636,76.956733,100.0,88.991533,94.495766
